# MODELAGEM MULTINÍVEIS E MARGINAIS

Nesse projeto usamos o dataset NHANES, uma coleção que aplica os valores armazenados em um banco de dados médicos para a análise dos dados, cada uma das variáveis é possivel encontrar o seu significado no link a seguir: https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDEDUC2

Iremos aplicar a temática de regressão modelagem multiníveis e marginais para análise dos nossos dados, os dados obtidos são do ano de 2015

In [5]:
# Importando os pacotes a serem utilizados
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [6]:
# Vamos agora importar os dados do NHANES do qual está em csv
df = pd.read_csv('C://Users//HENRIQUE//Documents//nhanes_2015_2016.csv')

In [7]:
df.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [9]:
# Vamos primeiramente realizar a limpeza dos nossos dados e deixar apenas as colunas que serão usadas
colunas = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
df = df[colunas].dropna()

In [11]:
df.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU
0,128.0,62,1,3,5.0,27.8,1,125,1
1,146.0,53,1,3,3.0,30.8,1,125,1
2,138.0,78,1,3,3.0,28.8,1,131,1
3,132.0,56,2,3,5.0,42.4,2,131,1
4,100.0,42,2,4,4.0,20.3,2,126,2


## Clusterização NHANES
O dataset foi coletado de condados diferentes dentro dos Estados Unidos e então selecionado subregiões e então selecionado pessoas dentro de cada uma dessas subregiões, é esperado que pessoas de um mesmo condado são mais similares do que pessoas de outros condados

Se nós pudessemos obter o condado id onde cada participante do NHANES reside, nós poderiamos estudar diretamente a estrutura de clusterização, porém por questão de serem informações pessoais elas não são divulgadas, nós então usamos subregiões combinadas de diferentes condados em grupos artificiais e que não são regiões geográficas deliberadamete contínuas, estes são chamados de MVU's(Masked Variance units), eles são deliberadamente selecionadas para tentar simular estas questões, portanto iremos explorar o MVU como clusters e explorar as extensões de cada variável do NHANES.

### Identificador MVU 
Para conseguir o nosso identificador MVU podemos obter pela combinação das colunas SDMVSTRA e SDMVPSU, do qual são identificadores regionais que iremos calcular a seguir: 

In [12]:
df['grupo'] = 10*df.SDMVSTRA + df.SDMVPSU

In [13]:
df.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,grupo
0,128.0,62,1,3,5.0,27.8,1,125,1,1251
1,146.0,53,1,3,3.0,30.8,1,125,1,1251
2,138.0,78,1,3,3.0,28.8,1,131,1,1311
3,132.0,56,2,3,5.0,42.4,2,131,1,1311
4,100.0,42,2,4,4.0,20.3,2,126,2,1262


### CORRELAÇÃO INTRACLASSE
Podemos fazer as nossas medições usando uma propriedade estatística chamada correlação intraclasse, ou ICC. Isto é uma forma distinta da correlação de Pearson. O iCC pega um valor de 0 a 1, do qual o 1 corresponde a uma perfeita clusterização, os valores com um cluster são identicos e 0 corresponde a uma independencia perfeita, o valor médio para cada cluster é identico sobre todos os cluster.

Nós podemos acessar o ICC usando duas regressões técnicas, regressão multiníveis e regressão marginal.

Para isso iremos iniciar usando uma técnica chamada GEE (Generalized Estimating Equations) para encaixar o nosso modelo linear marginal e estimar o ICC para os clusters do nosso dataset NHANES.

In [14]:
modelo = sm.GEE.from_formula('BPXSY1 ~ 1', groups = 'grupo', cov_struct = sm.cov_struct.Exchangeable(), data = df)
resultado = modelo.fit()
print(resultado.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


#### Análise do resultado
A estimativa do ICC é 0.03, do qual é pequena mas não negligenciável. Apesar de seu valor não ser diretamente comparável com o valor da correlação de pearson. Enqaunto 0.03 poderia ser considerado um valor pequeno se fosse uma variável da correlação de Pearson, isto não é pequeno para um ICC.

Nós claculamos o ICC como um número de variáveis diferentes que aparece em nossa análise, ou como resultado ou como preditor.

#### Vamos recodificar a coluna de fumantes (SMQ020) para uma variável binária simples
- Quando a pessoa não for fumante vamos atribuir valor igual a 0, diferente do que está na tabela como 2
- Valores como 7 e 9, que não condiz com as nossas variáveis, vamos atribuir como nulo(nan)

In [15]:
df['smq'] = df.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

In [16]:
df.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,grupo,smq
0,128.0,62,1,3,5.0,27.8,1,125,1,1251,1.0
1,146.0,53,1,3,3.0,30.8,1,125,1,1251,1.0
2,138.0,78,1,3,3.0,28.8,1,131,1,1311,1.0
3,132.0,56,2,3,5.0,42.4,2,131,1,1311,0.0
4,100.0,42,2,4,4.0,20.3,2,126,2,1262,0.0


#### Vamos portatno analisar a correlação em um mesmo cluster, ou seja uma análise intraclasse abaixo:

In [17]:
for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    modelo = sm.GEE.from_formula(v + ' ~ 1', groups = 'grupo', cov_struct = sm.cov_struct.Exchangeable(), data = df)
    resultado = modelo.fit()
    print(v, resultado.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026
SDMVSTRA The correlation between two observations in the same cluster is 0.959


### MODELO MARGINAL LINEAR COM DATA DEPENDENTE
Acima nó realizamos o calculo realizando uma quantificação induzida pela clusterização.

Uma outra faceta de trabalhar com data dependente é que podemos estimar sem considerar a dependência estrutural da data, o erro padrão e outras estatísticas relacionadas a incertezas vão estar erradas quando nós ignoramos a dependencia no nosso dataframe

Para ilustrar isso, iremos aplicar em dois modelos abaixo com uma mesma estrutura média para o NHANES. O primeiro é utilizar o mínimo quadrado ordinário, o segundo é usar o GEE, do qual permite considerar a dependecia na data.

#### Para ficar mais visual e facil de aplicar vamos criar uma nova coluna transformando os nossos dados
- 1: Masculino
- 2: Feminino

In [19]:
df["RIAGENDRx"] = df.RIAGENDR.replace({1: "Masculino", 2: "Feminino"})

In [21]:
df.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,grupo,smq,RIAGENDRx
0,128.0,62,1,3,5.0,27.8,1,125,1,1251,1.0,Masculino
1,146.0,53,1,3,3.0,30.8,1,125,1,1251,1.0,Masculino
2,138.0,78,1,3,3.0,28.8,1,131,1,1311,1.0,Masculino
3,132.0,56,2,3,5.0,42.4,2,131,1,1311,0.0,Feminino
4,100.0,42,2,4,4.0,20.3,2,126,2,1262,0.0,Feminino


#### Aplicar o modelo linear com o OLS - Ordinary least squares (mínimo quadrado ordinário)

In [22]:
modelo1 = sm.OLS.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)", data=df)
resultado1 = modelo1.fit()

#### Aplicar o modelo linear com o GEE

In [24]:
modelo2 = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
                              groups = 'grupo',
                              cov_struct = sm.cov_struct.Exchangeable(),
                              data=df)
resultado2 = modelo2.fit()

#### Vamos realizar a comparação dos nossos dois modelos, tanto os parâmetros, quanto nosso SE - Standard Error (Erro Padrão)

In [25]:
matriz_comparacao = pd.DataFrame({'OLS_params': resultado1.params, 'OLS_SE': resultado1.bse,
                                  'GEE_params': resultado2.params, 'GEE_SE': resultado2.bse})
matriz_comparacao = matriz_comparacao[['OLS_params', 'OLS_SE', 'GEE_params', 'GEE_SE']]
print(matriz_comparacao)

                        OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept                91.736583  1.339378   92.168530  1.384309
RIAGENDRx[T.Masculino]    3.671294  0.453763    3.650245  0.454498
C(RIDRETH1)[T.2]          0.855488  0.819486    0.159296  0.767025
C(RIDRETH1)[T.3]         -1.796132  0.671954   -2.233280  0.760228
C(RIDRETH1)[T.4]          3.813314  0.732355    3.105654  0.881580
C(RIDRETH1)[T.5]         -0.455347  0.808948   -0.439831  0.813675
RIDAGEYR                  0.478699  0.012901    0.474101  0.018493
BMXBMI                    0.278015  0.033285    0.280205  0.038553


#### Análise do nosso resutado
Podemos ver que os valores dos parâmetros são todos bem próximosdos dois modelos, porém vemos também que o modelo GEE tende a apresentar maior erro padrão.

O modelo GEE estimado e o erro padrão é significativo na presença de dependência contanto que a dependencia seja exclusivamente entre observações do mesmo cluster.

### MODELO MARGINAL LOGÍSTICO COM DATA DEPENDENTE
Abaixo iremos aplicar GEE em um modelo marginal linear na presença de dependencia.
GEE pode também ser usada para encaixar qualquer GLM na presença de dependencia.

Nós iremos mostrar a sua aplicação usando modelos do histórico de fumantes, igual fizemos anteriormente, porém dessa vez iremos aplicar o modelo GLM e depois aplicar o modelo marginal usando o GEE, porém uma coisa que devemos enfatizar é que GLM e GEE são ambos estimadores legítimos, porém o GLM não irá dar valores corretos do nosso erro padrão.

In [27]:
# Vamos agora renomear as nossa categoria escolares
df["DMDEDUC2x"] = df.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

# Aplicando os nossos dados em um modelo básico GLM
modelo1 = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           family=sm.families.Binomial(), data=df)
resultado1 = modelo1.fit()
resultado1.summary()

# Aplicando um modelo GLM usando GEE
modelo2 = sm.GEE.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           groups="grupo", family=sm.families.Binomial(),
           cov_struct=sm.cov_struct.Exchangeable(), data=df)
resultado2 = modelo2.fit(start_params=resultado1.params)

x = pd.DataFrame({"OLS_params": resultado1.params, "OLS_SE": resultado1.bse,
                  "GEE_params": resultado2.params, "GEE_SE": resultado2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                             OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept                     -2.305999  0.114308   -2.249820  0.140567
RIAGENDRx[T.Masculino]         0.909597  0.060167    0.908682  0.062342
C(DMDEDUC2x)[T.HS]             0.943364  0.089663    0.887965  0.095397
C(DMDEDUC2x)[T.SomeCollege]    0.832227  0.084361    0.771636  0.104449
C(DMDEDUC2x)[T.lt9]            0.266228  0.109183    0.321784  0.141327
C(DMDEDUC2x)[T.x9_11]          1.098561  0.106697    1.062149  0.138401
RIDAGEYR                       0.018257  0.001725    0.017416  0.001803


#### Análise dos resultados
Assim como esperado, ambos os modelos dão resultados muito similares para os parâmetros, entretanto o valor do desvio padrão obtido usando o GEE é de alguma forma maior do que aquele obtido no GLM, isso é diretamente da consequência de ignorar a estimativa da estrutura de dependência.
O GEE trabalha em 3 fatores:
- GEE nos dá uma visão sobre a estrutura de dependência dos dados
- O GEE usa a estrutura de dependência para obter erros padrão significativos dos parâmetros estimados do modelo.
- GEE usa a estrutura de dependência para estimar os parâmetros do modelo com mais precisão

#### Conclusão
Portanto podemos compreender que o GEE possui uma vantagem de eficiência um pouco melhor do que o GLM, pois o GLM nos dá uma visão mais otimista e o GLM não nos da a visão do primeiro tópico acima, que é a visão sobre a estrutura de dependência dos dados

## MODELO MULTINÍVEIS
O modelo de muliníveis é uma forma deaplicação para o modelo trabalhado acima, ou seja de regressão marginal, portatno o modelo de multiníveis é similar ao modelo marginal estimando o mesmo grupo alvo, mas apresente esse nosso alvo de diferentes formas e utiliza procedimentos diferentes para chegar no resultado.

Nós podemos imaginar que cada cluster tem um efeito aleatório que é dividido por todas as observações no nosso cluster.

#### Aplicando um modelo de multiníveis para suportar data dependente

In [29]:
modelo = sm.MixedLM.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="grupo", data=df)
resultado = modelo.fit()
resultado.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   BPXSY1     
No. Observations:     5102      Method:               REML       
No. Groups:           30        Scale:                256.6952   
Min. group size:      106       Log-Likelihood:       -21409.8702
Max. group size:      226       Converged:            Yes        
Mean group size:      170.1                                      
-----------------------------------------------------------------
                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------
Intercept              92.173    1.402 65.752 0.000 89.426 94.921
RIAGENDRx[T.Masculino]  3.650    0.452  8.084 0.000  2.765  4.535
C(RIDRETH1)[T.2]        0.153    0.887  0.172 0.863 -1.586  1.891
C(RIDRETH1)[T.3]       -2.238    0.758 -2.954 0.003 -3.723 -0.753
C(RIDRETH1)[T.4]        3.098    0.836  3.707 0.000  1.460  4.737
C(RIDRETH1)[T.5]       -0.439    0.878 -0.500 0.617 -2.161  1.282
RIDAGEYR                0.474    0.013 36.482 0.000  0.449  0.500
BMXBMI                  0.280    0.033  8.404 0.000  0.215  0.346
grupo Var               3.615    0.085                           
=================================================================

"""

#### Análise dos resultados
A variância de parâmetros estruturais é oque dinstingue um modelo msito de um modelo marginal, aqui no caso acima temos um parâmetro do qual a variância para os grupos sejam estimados a ser 3.615. Isto significa que os efeitos aleatórios de dois grupos escolhidos seriam 2.69, ou seja, a raiz quadrada de (2 * 3.615).

O modelo misto tem as mesmas propriedades desejadas pelo GEE, em fato cada um desses dois métodos é muito útil e amplamente utilizada, e em ambos existem certas peculiaridades que podem ser mias vantajosas entre um modelo ou outro.

O modelo multinível pode ser utilizado estimando o valor do ICC. No caso do modelo com um nível que é o que temos aqui, o ICC é a variância da nossa variável de grupo (3.615) dividido pela soma da variância da variável de grupo e das variâncias inexplicadas (256.7).

### EFEITOS ALEATÓRIOS PREVISTOS
Enqaunto os efeitos aleatórios em um modelo multinível ainda não foi observado, nós podemos prever eles de nosso dataset.

No nosso NHANES dataset por exemplo, nós poderíamos prever os eventos aleatórios para quantificar a unicidade de cada condado relativo a média.

Podemos visualizar os nossos efeitos aleatórios para os 30 grupos nas análises que são mostradas abaixo:

In [30]:
resultado.random_effects

{1191: grupo   -1.630976
 dtype: float64,
 1192: grupo   -0.086162
 dtype: float64,
 1201: grupo   -2.042661
 dtype: float64,
 1202: grupo   -0.147472
 dtype: float64,
 1211: grupo    0.280623
 dtype: float64,
 1212: grupo    1.580732
 dtype: float64,
 1221: grupo    0.283347
 dtype: float64,
 1222: grupo    0.131512
 dtype: float64,
 1231: grupo   -2.038171
 dtype: float64,
 1232: grupo    0.617651
 dtype: float64,
 1241: grupo    2.878488
 dtype: float64,
 1242: grupo   -0.519364
 dtype: float64,
 1251: grupo    2.064967
 dtype: float64,
 1252: grupo    1.521281
 dtype: float64,
 1261: grupo   -1.261975
 dtype: float64,
 1262: grupo    0.980846
 dtype: float64,
 1271: grupo    0.118031
 dtype: float64,
 1272: grupo   -0.128397
 dtype: float64,
 1281: grupo   -0.384862
 dtype: float64,
 1282: grupo   -3.582111
 dtype: float64,
 1291: grupo   -3.271017
 dtype: float64,
 1292: grupo   -0.829538
 dtype: float64,
 1301: grupo   -0.884171
 dtype: float64,
 1302: grupo    2.790657
 dtype: f

#### Análise dos resultados
Podemos verificar que os efeitos aleatórios, por exemplo no cluster 1241 teve um alto SBP incomum e o cluster 1282 teve um baixo SBP incomum.

Esses desvios do que é esperado são observados depois do ajuste para as covariações no modelo, e consequentemente são ocasionados por características que não são refletidas pelas nossas covariações.

### 'SLOPES' ALEATÓRIOS
Nós devemos aplicar duas variações neste modelo. No primeiro modelo, o cluster específica que os nossos 'intercepts'(onde corta o eixo y) e slopes são variáveis independentes.

É recomendado centralizar qualquer covariável que tem um 'slope' aleatório. Isto resulta em modelos que convergem rapidamente e são mais robustos, portatno devemos centralizar com cada cluster, embora seja comum também realizar a centralização do nosso dataset inteiro, ao invés de clusters separadamente.

In [31]:
df['age_cen'] = df.groupby('grupo').RIDAGEYR.transform(lambda x: x - x.mean())
modelo = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)",
                                groups = 'grupo', vc_formula = {'age_cen': '0+age_cen'}, data = df)
resultado = modelo.fit()
resultado.summary()

C:\Users\HENRIQUE\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
====================================================================
Model:                 MixedLM    Dependent Variable:    BPXSY1     
No. Observations:      5102       Method:                REML       
No. Groups:            30         Scale:                 263.7323   
Min. group size:       106        Log-Likelihood:        -21469.9240
Max. group size:       226        Converged:             Yes        
Mean group size:       170.1                                        
--------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------
Intercept              115.207    1.209 95.265 0.000 112.836 117.577
RIAGENDRx[T.Masculino]   3.643    0.457  7.962 0.000   2.746   4.539
C(RIDRETH1)[T.2]         1.167    0.827  1.412 0.158  -0.453   2.787
C(RIDRETH1)[T.3]        -1.659    0.679 -2.444 0.015  -2.989  -0.328
C(RIDRETH1)[T.4]         3.610    0.739  4.884 0.000   2.161   5.058
C(RIDRETH1)[T.5]        -1.208    0.816 -1.480 0.139  -2.807   0.392
age_cen                  0.467    0.018 26.235 0.000   0.432   0.502
BMXBMI                   0.288    0.034  8.574 0.000   0.222   0.353
age_cen Var              0.004    0.000                             
====================================================================

"""

#### Análise dos resultados
Nós podemos estimar a variação aleatória dos 'slopes' na nossa aba age(idade) = 0.004,
do qual traduzimos para um desvio padrão de 0.06.

Portanto isso significa que de um cluster para outro ele pode variar de +-0.06 - 0.12, ou seja, podemos ter valores no intervalo de:
- 0.467 + 0.06 = 0.527 mm/Hg
- 0.467 - 0.06 = 0.407 mm/Hg